# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? 

Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
# import spark
from pyspark.sql import SparkSession

In [2]:
# create a spark session
spark = SparkSession.builder.appName('log_reg_project').getOrCreate()

In [3]:
# load the data
data = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)

In [4]:
# print the schema
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [5]:
# count how many rows of data there are
data.count()

900

In [6]:
# print the first 5 rows of data
data.head(5)

[Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1),
 Row(Names='Kevin Mueller', Age=41.0, Total_Purchase=11916.22, Account_Manager=0, Years=6.5, Num_Sites=11.0, Onboard_date=datetime.datetime(2013, 8, 13, 0, 38, 46), Location='6157 Frank Gardens Suite 019 Carloshaven, RI 17756', Company='Wilson PLC', Churn=1),
 Row(Names='Eric Lozano', Age=38.0, Total_Purchase=12884.75, Account_Manager=0, Years=6.67, Num_Sites=12.0, Onboard_date=datetime.datetime(2016, 6, 29, 6, 20, 7), Location='1331 Keith Court Alyssahaven, DE 90114', Company='Miller, Johnson and Wallace', Churn=1),
 Row(Names='Phillip White', Age=42.0, Total_Purchase=8010.76, Account_Manager=0, Years=6.71, Num_Sites=10.0, Onboard_date=datetime.datetime(2014, 4, 22, 12, 43, 12), Location='13120 Daniel Mount Angelabury, WY 30645-4695'

In [7]:
# describe the data
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [8]:
# see column names
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [9]:
# import the necessary libraries
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression

# assemble the features into a single vector
assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase', 
                                    'Account_Manager', 'Years', 
                                    'Num_Sites'], outputCol='features')

In [10]:
# transform the data
output = assembler.transform(data)

In [11]:
# get the final data
final_data = output.select('features', 'churn')

### Do Train-trest-split

In [12]:
# split the data into training and testing sets
train_churn, test_churn = final_data.randomSplit([0.7, 0.3])

In [13]:
# create a logistic regression model
lr_churn = LogisticRegression(labelCol='churn')

In [14]:
# fit the model to the training data
fit_churn_model = lr_churn.fit(train_churn)

In [15]:
# see the summary of the model
training_summary = fit_churn_model.summary

training_summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                604|                604|
|   mean|0.17052980132450332| 0.1291390728476821|
| stddev|0.37640930419496493|0.33563175824716124|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



### Evaluate the results

In [16]:
# get the predictions on the test data
test_results = fit_churn_model.evaluate(test_churn)

In [17]:
# get the predictions and labels
pred_and_labels = test_results.predictions.select('prediction', 'churn')

In [18]:
# show the predictions and labels
pred_and_labels.show()

+----------+-----+
|prediction|churn|
+----------+-----+
|       0.0|    0|
|       0.0|    0|
|       0.0|    1|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       1.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
|       0.0|    0|
+----------+-----+
only showing top 20 rows



In [19]:
# use AUC to evaluate the model
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# create an evaluator
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                             labelCol='churn')

# get the AUC
auc = churn_eval.evaluate(test_results.predictions)
auc

0.7844142527557036

### Predicting on brand new unlabelled data

In [20]:
# read in the new customer data
new_customers = spark.read.csv('new_customers.csv', inferSchema=True, header=True)

In [21]:
# show the data
new_customers.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|
|  Jeremy Chang|65.0|         100.0|              1|  1.0|     15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|
|Megan Ferguson|32.0|        6487.5|              0|  9.4|     14.0|2016-10-28 05:32:13|922 Wright Branch...|   Sexton-Golden|
|  Taylor Young|32.0|      13147.71|              1| 10.0|      8.0|2012-03-20 00:36:46|Unit 0789 Box 073...|  

In [22]:
# fit final model to all the data
final_lr_model = lr_churn.fit(final_data)

In [23]:
# transform the new customers data
new_customers_assembler = assembler.transform(new_customers)

In [24]:
# check schema
new_customers_assembler.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [25]:
# get the final results
final_results = final_lr_model.transform(new_customers_assembler)

In [26]:
# show the resulta of company predictions
final_results.select('Company', 'prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



We should assign account managers to Cannon-Benson, Barron-Robertson, Sexton-Golden and Parks-Robbins.

In [27]:
# evaluate which features are most important
final_lr_model.coefficients

# get the feature names
assembler.getInputCols()

# zip the coefficients and feature names together
coeff_summary = zip(assembler.getInputCols(), final_lr_model.coefficients)

# print the coefficients
for item in coeff_summary:
    print(item)


('Age', 0.05980689744733625)
('Total_Purchase', 3.731291275333842e-05)
('Account_Manager', 0.48095367061632205)
('Years', 0.5659857803672689)
('Num_Sites', 1.2115730289321005)
